# A Markov chain Monte Carlo (MCMC) Bayesian inference approach to analyze apparent activation barriers and reaction orders from microreactor data GUI

In [11]:
# Python packages for Bayesian inference, linear regression, and PySimpleGui

# In your preferred conda environment, install the following Python packages for the Jupyter Notebook to work:
# 1) PySimpleGui
# 2) pymc
# 3) sklearn

import PySimpleGUI as sg
import os
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import pytensor.tensor as pt
import xarray as xr

from sklearn.linear_model import LinearRegression

np.set_printoptions(suppress=True)

In [12]:
# Setting parameters for output figures to be of "publishable" quality for the linear regression plots

def regression_plots():
    az.style.use("default")
    params = {'axes.labelsize': 35,             # Default: 'medium'
              'savefig.bbox': 'tight',
              'figure.titlesize': 'large',      # Default: 'large'
              'legend.fontsize': 15,            # Default: 'medium'
              'lines.linewidth': 3,           # Default: 1.5
              'lines.markersize': 15,           # Default: 6
              'font.weight': 'normal',          # Default: 'normal'
              'xtick.labelsize': 25,            # Default: 'medium'
              'ytick.labelsize': 25,            # Default: 'medium'
              'axes.linewidth': 3.5,            # Default: 0.8
              'axes.labelweight': 'normal',     # Default: 'normal'
    #           'font.family': 'Arial',           # Default: 'sans-serif'
              'font.size': 5,                  # Default: 10
              'legend.fancybox': False,
              'legend.edgecolor': 'black',
              'ytick.major.pad': 10,
              'xtick.major.pad': 10,
              'xtick.major.size': 10,            # Default: 3.5    
              'xtick.major.width': 2,
              'ytick.major.size': 10,
              'ytick.major.width': 2
             }

    plt.rcParams.update(params)
    
    return None

In [13]:
# Setting parameters for output figures to be of "publishable" quality for the traces of the MCMC

def trace_plots():
    %config InlineBackend.figure_format = 'retina'
    az.style.use("arviz-darkgrid")
    
    return None

In [14]:
# Linear regression analysis of the experimental data to obtain the mean for the priors of the slope and intercept

def least_squares_regression(data):
    data_numpy = data.to_numpy(copy=True)
    
    ln_P = data_numpy[:, 1]
    ln_rate = data_numpy[:, 2]
    
    slope, intercept = np.polyfit(ln_P, ln_rate, 1)
    
    return slope, intercept

In [15]:
# Bayesian inference with all experimental data as one dataset

def complete_pooling(data, slope_mean, intercept_mean, slope_std, intercept_std, ln_rate_std, x_label, y_label):  
    
    columns = list(data.columns)
    
    trials, trial_number =data.trial.factorize()
    
    data_numpy = data.to_numpy(copy=True)
    
    ln_P_values = data_numpy[:,1]
    ln_r_values = data_numpy[:,2]

    with pm.Model() as pooled_model:
        ln_P = pm.MutableData(x_label, ln_P_values, dims="obs_id")
        
        # Priors
        slope = pm.Normal("slope", mu=slope_mean, sigma=np.abs(slope_std))
        intercept = pm.Normal("intercept", mu=intercept_mean, sigma=np.abs(intercept_std))
        
        # Likelihood
        ln_r = slope * ln_P + intercept
        
        y = pm.Normal(y_label, ln_r, sigma=ln_rate_std, observed=ln_r_values, dims="obs_id")
        
    with pooled_model:
        step = pm.NUTS()
        pooled_trace = pm.sample(step=step, tune=5000, draws=20000)
        pm.sample_posterior_predictive(pooled_trace, extend_inferencedata=True)
        
    pooled_df = az.summary(pooled_trace, round_to=2)
    headers = pooled_df.columns.to_list()[0:4]
    headers.insert(0,"")
    values = pooled_df.to_records().tolist()
    
    trace_plots()
    
    pooled_trace_plot = az.plot_trace(pooled_trace)
    fig = pooled_trace_plot.ravel()[0].figure;
    
    # How to save PNGs to the same directory where this script is located
    fig.savefig("complete_pooling_trace.png")

    post = pooled_trace.posterior
    mu_pp = post["intercept"] + post["slope"] * xr.DataArray(ln_P_values, dims=["obs_id"])
    
    regression_plots()
    
    _, ax = plt.subplots(figsize=(8,8))
    
    if intercept_mean > 0:
        label_regression = "%s = %0.2f * %s + %0.2f " % (y_label, values[0][1], x_label, values[1][1])
    else:
        label_regression = "%s = %0.2f * %s - %0.2f " % (y_label, values[0][1], x_label, abs(values[1][1]))

    ax.plot(ln_P_values, mu_pp.mean(("chain", "draw")), label=label_regression, color="C1", alpha=0.6)

    ax.scatter(ln_P_values, pooled_trace.observed_data[y_label], color='blue')
    az.plot_hdi(ln_P_values, pooled_trace.posterior_predictive[y_label], hdi_prob=0.94)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.legend(frameon=False, fontsize=20, bbox_to_anchor=(1, 1))
    _.savefig("complete_pooling_hdi.png")
    
    # Generating an output window containing a plot of the linear regression and a table of slope and intercept with their 
    # respective means, standard deviation, and 94% HDI's
    col2 = [[sg.Image("complete_pooling_hdi.png")],
            [sg.Table(values=values, headings=headers, expand_x=True, justification='center', hide_vertical_scroll=True)]]
    
    layout2 = [[sg.Column(col2, scrollable=True, vertical_scroll_only=False, size=(1500,1500))]]

    window2 = sg.Window("Complete Pooling", layout2, finalize=True, resizable=True)
    
    # Generating another output window containing the trace of the MCMC
    layout3 = [[sg.Image("complete_pooling_trace.png")]]
    
    window3 = sg.Window("Complete Pooling Trace", layout3, finalize=True)
    
    while True:
        window4, event, values = sg.read_all_windows()
        if event == sg.WIN_CLOSED or event=="Exit":
            window4.close()
            if window4 == window3:
                window3 = None
            elif window4 == window2:
                break
    
    return None

In [16]:
# Bayesian inference constraining the slope

def vary_intercept(data, slope_mean, intercept_mean, slope_std, intercept_std, ln_r_std, x_label, y_label):  
   
    columns = list(data.columns)
    
    trials, trial_number =data.trial.factorize()
    
    coords = {"trials": trial_number}
    
    data_numpy = data.to_numpy(copy=True)
    
    ln_P_values = data_numpy[:,1]
    ln_r_values = data_numpy[:,2]
    
    with pm.Model(coords = coords) as vary_intercept_model:
        ln_P = pm.MutableData(x_label, ln_P_values, dims="obs_id")
        trial_index = pm.MutableData("trial_index", trials, dims="obs_id")

        # Priors 
        # Common Slope
        slope = pm.Normal("slope", mu=slope_mean, sigma=np.abs(slope_std))
        # Random Intercepts
        intercept = pm.Normal("intercept", mu=intercept_mean, sigma=np.abs(intercept_std), dims="trials")
        
        # Likelihood
        ln_r = slope * ln_P + intercept[trial_index]
        
        y = pm.Normal(y_label, ln_r, sigma = ln_r_std, observed=ln_r_values, dims="obs_id")
        
    with vary_intercept_model:
        step = pm.NUTS()
        vary_intercept_trace = pm.sample(step=step, tune=5000, draws=20000)
        
    vary_intercept_df = az.summary(vary_intercept_trace, round_to=2)
    headers = vary_intercept_df.columns.to_list()[0:4]
    headers.insert(0,"")
    values = vary_intercept_df.to_records().tolist()
    
    trace_plots()
    
    vary_intercept_trace_plot = az.plot_trace(vary_intercept_trace, figsize=(12,7))
    fig = vary_intercept_trace_plot.ravel()[0].figure
    fig.savefig("vary_intercept_trace.png")
    
    headers = vary_intercept_df.columns.to_list()[0:4]
    headers.insert(0,"")
    values = vary_intercept_df.to_records().tolist()
    
    post = vary_intercept_trace.posterior
    mu_pp = post["intercept"] + post["slope"] * xr.DataArray(ln_P_values, dims=["obs_id"])
    
    regression_plots()
    
    fig, ax = plt.subplots(figsize=(8,8))
    grouped_data = data.groupby(data.trial)
    for trial in trial_number:
        trial_data = grouped_data.get_group(trial)
        columns = list(trial_data.columns)
        ax.scatter(trial_data.iloc[:,1], trial_data.iloc[:,2])
    for trial in trial_number:
        trial_index = trial - 1
        if intercept_mean > 0:
            label_regression = "%s = %0.2f * %s + %0.2f " % (y_label, values[0][1], x_label, abs(values[trial][1]))
        else:
            label_regression = "%s = %0.2f * %s - %0.2f " % (y_label, values[0][1], x_label, abs(values[trial][1]))
        ax.plot(ln_P_values, mu_pp.mean(("chain", "draw"))[trial_index], label = label_regression)
    ax.legend(frameon=False, fontsize=20, bbox_to_anchor=(1, 1))
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    fig.savefig("vary_intercept_mean.png")
    
    col5 = [[sg.Image("vary_intercept_mean.png")],
            [sg.Table(values=values, headings=headers, expand_x=True, justification='center', hide_vertical_scroll=True)]]
    
    layout5 = [[sg.Column(col5, scrollable=True, vertical_scroll_only=False, size=(1500,1500))]]
    
    window5 = sg.Window("Vary Intercept Model", layout5, finalize=True, resizable=True)
    
    layout6 = [[sg.Image("vary_intercept_trace.png")]]
    
    window6 = sg.Window("Vary Intercept Trace", layout6, finalize=True)
    
    while True:
        window7, event, values = sg.read_all_windows()
        if event == sg.WIN_CLOSED or event=="Exit":
            window7.close()
            if window7 == window6:
                window6=None
            elif window7 == window5:
                break

    return None

In [17]:
# Bayesian inference for individual datasets

def vary_intercept_slope(data, slope_mean, intercept_mean, slope_std, intercept_std, ln_r_std, x_label, y_label):  
    print(slope_mean, intercept_mean, slope_std, intercept_std, ln_r_std)
    
    columns = list(data.columns)
    
    trials, trial_number =data.trial.factorize()
    
    coords = {"trials": trial_number}
    
    data_numpy = data.to_numpy(copy=True)
    
    ln_P_values = data_numpy[:,1]
    ln_r_values = data_numpy[:,2]
    
    with pm.Model(coords = coords) as vary_intercept_slope_model:
        ln_P = pm.MutableData("ln_P", ln_P_values, dims="obs_id")
        trial_index = pm.MutableData("trial_index", trials, dims="obs_id")

        
        # Priors 
        # Random Slope
        slope = pm.Normal("slope", mu=slope_mean, sigma=slope_std, dims="trials")
        # Random Intercepts
        intercept = pm.Normal("intercept", mu=intercept_mean, sigma=np.abs(intercept_std), dims="trials")
        
        # Likelihood
        ln_r = slope[trial_index] * ln_P + intercept[trial_index]
        
        y = pm.Normal("ln_r", ln_r, sigma = ln_r_std, observed=ln_r_values, dims="obs_id")
        
    with vary_intercept_slope_model:
#         step = pm.Metropolis()
        vary_intercept_slope_trace = pm.sample()        
    vary_intercept_slope_df = az.summary(vary_intercept_slope_trace, round_to=4)
    headers = vary_intercept_slope_df.columns.to_list()[0:4]
    headers.insert(0,"")
    values = vary_intercept_slope_df.to_records().tolist()
    
    trace_plots()
    
    vary_intercept_slope_trace_plot = az.plot_trace(vary_intercept_slope_trace, figsize=(12,7))
    fig = vary_intercept_slope_trace_plot.ravel()[0].figure
    fig.savefig("vary_intercept_slope_trace.png")

    post = vary_intercept_slope_trace.posterior
    mu_pp = post["intercept"] + post["slope"] * xr.DataArray(ln_P_values, dims=["obs_id"])
    
    regression_plots()
    
    fig, ax = plt.subplots(figsize=(8,8))
    grouped_data = data.groupby(data.trial)
    for trial in trial_number:
        trial_data = grouped_data.get_group(trial)
        columns = list(trial_data.columns)
        ax.scatter(trial_data.iloc[:,1], trial_data.iloc[:,2])
    for trial in trial_number:
        trial_index = trial - 1
        if intercept_mean > 0:
            label_regression = "%s = %0.2f * %s + %0.2f " % (y_label, values[trial_index][1], x_label, abs(values[trial_index+4][1]))
        else:
            label_regression = "%s = %0.2f * %s - %0.2f " % (y_label, values[trial_index][1], x_label, abs(values[trial_index+4][1]))
            
        ax.plot(ln_P_values, mu_pp.mean(("chain", "draw"))[trial_index], label = label_regression)
    ax.legend(frameon=False, fontsize=20, bbox_to_anchor=(1, 1))
    ax.set_xlabel("ln(P)")
    ax.set_ylabel("ln(r)")
    fig.savefig("vary_intercept_slope_mean.png")
    
    col4 = [[sg.Image("vary_intercept_slope_mean.png")],
            [sg.Table(values=values, headings=headers, expand_x=True, justification='center', hide_vertical_scroll=True)]]
    
    layout4 = [[sg.Column(col4, scrollable=True, vertical_scroll_only=False, size=(1500,1500))]]

    
    window4 = sg.Window("Vary Intercept and Slope Model", layout4)

    while True:
        event, values = window4.read()
        if event == sg.WIN_CLOSED or event=="Exit":
            break
    
    
    return None

In [ ]:
# Layout for the input window

layout = [[sg.Text("Choose formatted .CSV file containing the appropriate linear data to be modeled: "), 
           sg.Input(key="-INPUT1-"), 
           sg.FileBrowse(key="-IN1-")],
          [sg.Text("Enter label for the independent variable (x): "), sg.Input(key="-INPUT2-")],
          [sg.Text("Enter label for the dependent variable (y): "), sg.Input(key="-INPUT3-")],
          
          [sg.T("")],
          
          [sg.Text("Perform a Least Squares Regression analysis to obtain mean for the slope and intercept?"), 
           # Divide question to provide std dev but calculating mean (default value?)
           sg.Radio("Yes", "LSR", key="-IN2-", enable_events=True), 
           sg.Radio("No", "LSR", key="-IN3-", enable_events=True)],
          
          [sg.Text(size=(100,1), key="-OUTPUT1-", text_color="yellow")],
          
          [sg.Text("Provide standard deviations for the slope and intercept?"),
          sg.Radio("Yes", "StdDev", key="-IN15-", enable_events=True),
          sg.Radio("No", "StdDev", key="-IN16-", enable_events=True)],
          
          [sg.Text("Standard deviations for slope and intercept are numerical parameters which will affect the 94% HDI")],
          
          [sg.Text(size=(100,1), key='-OUTPUT2-', text_color="yellow")],
          
#           [sg.T("")],
          
          [sg.T(""), sg.Push(), sg.Text("        "), sg.Text("y"),sg.Text("   "), sg.Text("Slope"), sg.Text(""), sg.Text("Intercept"), sg.Push()],
          
          [sg.Push(), sg.Text("Mean"), sg.Text("              "), sg.Input(size=(10,1), key="-IN7-"), sg.Input(size=(10,1), key="-IN8-"), sg.Push()],
          
          [sg.Push(), sg.Text("St. Dev."), sg.Input(size=(10,1), key="-IN9-"), sg.Input(size=(10,1), key="-IN10-"), sg.Input(size=(10,1), key="-IN11-"), sg.Push()],
          
          [sg.Push(), sg.Text("Model to be Used:"), sg.Radio("Pooled", "Model", key="-IN12-"), sg.Radio("Vary Intercept", "Model", key="-IN13-"), sg.Radio("Vary Intercept and Slope", "Model", key="-IN14-"), sg.Push()],
#          [sg.Push(), sg.Text("Model to be Used:"), sg.Radio("Pooled", "Model", key="-IN12-"), sg.Radio("Vary Intercept", "Model", key="-IN13-"), sg.Push()],

                    
          [sg.T("")],
                    
          
          [sg.Push(), sg.Button("Run"), sg.Button("Exit"), sg.Push()]
                    
         ]

window = sg.Window("Bayesian GUI", layout)

while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event=="Exit":
        break
        
    if values["-IN3-"] == True:
        window["-OUTPUT1-"].update("If no, you will be required to provide the mean for the slope and intercept.")
        window["-IN7-"].update(disabled=False)
        window["-IN8-"].update(disabled=False)
    elif (values["-IN2-"] == True):
        window["-OUTPUT1-"].update("")
        window["-IN7-"].update(disabled=True)
        window["-IN8-"].update(disabled=True)

    if values["-IN15-"] == True:
        window["-OUTPUT2-"].update("")
        window["-IN10-"].update(disabled=False)
        window["-IN11-"].update(disabled=False)
    elif values["-IN16-"] == True:
        window["-OUTPUT2-"].update("If no, the standard deviation will be assumed to be 10% of the mean.")
        window["-IN10-"].update(disabled=True)
        window["-IN11-"].update(disabled=True)
        
    if event == "Run":
        data = pd.read_csv(values["-INPUT1-"])
        
        ln_rate_std = float(values["-IN9-"])
        
        if values["-IN2-"] == True:
            slope, intercept = least_squares_regression(data)
        else:
            slope = float(values["-IN7-"])
            intercept = float(values["-IN8-"])            
        
        if values["-IN15-"] == True:
            slope_std = float(values["-IN10-"])
            intercept_std = float(values["-IN11-"])
        else:
            slope_std = slope * 0.1
            intercept_std = intercept * 0.1
            
        if values["-IN12-"] == True:
#             print("Complete Pooled Model")
            model = "Complete"
            complete_pooling(data, slope, intercept, slope_std, intercept_std, ln_rate_std, values["-INPUT2-"], values["-INPUT3-"])       
        elif values["-IN13-"] == True:
#             print("Vary Intercept Model")
            model = "Vary Intercept"
            vary_intercept(data, slope, intercept, slope_std, intercept_std, ln_rate_std, values["-INPUT2-"], values["-INPUT3-"])
        
        elif values["-IN14-"] == True:
#            print("Vary Intercept and Slope Model")
            model = "Vary Intercept Slope"
            vary_intercept_slope(data, slope, intercept, slope_std, intercept_std, ln_rate_std, values["-INPUT2-"], values["-INPUT3-"])
        
window.close()